## IMPORT

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 462 kB 16.5 MB/s 
     |████████████████████████████████| 4.2 MB 6.4 MB/s 


In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [3]:
cd /content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0 

/content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0


In [4]:
print(os.getcwd())

/content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0


## Preparing the dataset

In [5]:
import userFeatures
builder = tfds.builder('Userfeatures')
userFeatureDs = tfds.load('Userfeatures',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userfeatures/1.0.0.incompleteG9MWZ4/userfeatures-train.tfrecord*...:   0%|…

Dataset userfeatures downloaded and prepared to /root/tensorflow_datasets/userfeatures/1.0.0. Subsequent calls will reuse this data.


In [6]:
import wishEmbedding
builder = tfds.builder('Wishembedding')
wishEmbeddingDs = tfds.load('Wishembedding',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishembedding/1.0.0.incompletePU2ULS/wishembedding-train.tfrecord*...:   0…

Dataset wishembedding downloaded and prepared to /root/tensorflow_datasets/wishembedding/1.0.0. Subsequent calls will reuse this data.


In [11]:
ratings = userFeatureDs.map(lambda x: {
    "location_name": x["location_name"],
    "user_id": x["user_id"],
    "add" : x["add"]
})
locations = wishEmbeddingDs.map(lambda x: x["location_name"])

In [13]:
for x in ratings.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'add': 0, 'location_name': b'JAKARTA', 'user_id': b'93'}
{'add': 0, 'location_name': b'DUBAI', 'user_id': b'112'}


In [19]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(NUM_DATA, seed=42, reshuffle_each_iteration=False)

NUM_DATA = ratings.__len__().numpy()

trainset_size = 0.8 * NUM_DATA

train = shuffled.take(trainset_size)
test = shuffled.skip(trainset_size).take(NUM_DATA - trainset_size)

location_name = locations.batch(1000)
user_ids = ratings.batch(1000).map(lambda x: x["user_id"])

unique_location_name = np.unique(np.concatenate(list(location_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## MODEL

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In [36]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(128).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_name"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add")

    user_embeddings, location_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [37]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [38]:
cached_train = train.shuffle(NUM_DATA).batch(512).cache()
cached_test = test.batch(256).cache()

In [39]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
43/43 [==============================] - 5s 78ms/step - root_mean_squared_error: 0.4578 - factorized_top_k/top_1_categorical_accuracy: 0.0026 - factorized_top_k/top_5_categorical_accuracy: 0.0363 - factorized_top_k/top_10_categorical_accuracy: 0.0781 - factorized_top_k/top_50_categorical_accuracy: 0.4176 - factorized_top_k/top_100_categorical_accuracy: 0.8224 - loss: 0.2091 - regularization_loss: 0.0000e+00 - total_loss: 0.2091
Epoch 2/3
43/43 [==============================] - 4s 82ms/step - root_mean_squared_error: 0.4462 - factorized_top_k/top_1_categorical_accuracy: 0.0044 - factorized_top_k/top_5_categorical_accuracy: 0.0388 - factorized_top_k/top_10_categorical_accuracy: 0.0821 - factorized_top_k/top_50_categorical_accuracy: 0.4189 - factorized_top_k/top_100_categorical_accuracy: 0.8271 - loss: 0.1990 - regularization_loss: 0.0000e+00 - total_loss: 0.1990
Epoch 3/3
22/22 [==============================] - 3s 75ms/step - root_mean_squared_error: 0.4463 - factorized_top_k

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [40]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [42]:
model.fit(cached_train, epochs=5)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/5
43/43 [==============================] - 4s 103ms/step - root_mean_squared_error: 0.5020 - factorized_top_k/top_1_categorical_accuracy: 0.3450 - factorized_top_k/top_5_categorical_accuracy: 0.5509 - factorized_top_k/top_10_categorical_accuracy: 0.6474 - factorized_top_k/top_50_categorical_accuracy: 0.8881 - factorized_top_k/top_100_categorical_accuracy: 0.9862 - loss: 3175.7363 - regularization_loss: 0.0000e+00 - total_loss: 3175.7363
Epoch 2/5
43/43 [==============================] - 4s 85ms/step - root_mean_squared_error: 0.5023 - factorized_top_k/top_1_categorical_accuracy: 0.3821 - factorized_top_k/top_5_categorical_accuracy: 0.6556 - factorized_top_k/top_10_categorical_accuracy: 0.6932 - factorized_top_k/top_50_categorical_accuracy: 0.9219 - factorized_top_k/top_100_categorical_accuracy: 0.9894 - loss: 3175.6546 - regularization_loss: 0.0000e+00 - total_loss: 3175.6546
Epoch 3/5
43/43 [==============================] - 3s 80ms/step - root_mean_squared_error: 0.5029 - fac

In [43]:
model.evaluate(cached_test, return_dict=True)

22/22 [==============================] - 4s 157ms/step - root_mean_squared_error: 0.5036 - factorized_top_k/top_1_categorical_accuracy: 0.0060 - factorized_top_k/top_5_categorical_accuracy: 0.5929 - factorized_top_k/top_10_categorical_accuracy: 0.6654 - factorized_top_k/top_50_categorical_accuracy: 0.8253 - factorized_top_k/top_100_categorical_accuracy: 0.9749 - loss: 1344.4653 - regularization_loss: 0.0000e+00 - total_loss: 1344.4653


{'factorized_top_k/top_100_categorical_accuracy': 0.9748634099960327,
 'factorized_top_k/top_10_categorical_accuracy': 0.6653916239738464,
 'factorized_top_k/top_1_categorical_accuracy': 0.006010929122567177,
 'factorized_top_k/top_50_categorical_accuracy': 0.8253187537193298,
 'factorized_top_k/top_5_categorical_accuracy': 0.5928961634635925,
 'loss': 540.0950317382812,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.5035766959190369,
 'total_loss': 540.0950317382812}

### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [25]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [26]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
43/43 [==============================] - 5s 85ms/step - root_mean_squared_error: 0.4567 - factorized_top_k/top_1_categorical_accuracy: 2.7322e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0211 - factorized_top_k/top_10_categorical_accuracy: 0.0523 - factorized_top_k/top_50_categorical_accuracy: 0.3571 - factorized_top_k/top_100_categorical_accuracy: 0.7973 - loss: 3181.1915 - regularization_loss: 0.0000e+00 - total_loss: 3181.1915
Epoch 2/3
43/43 [==============================] - 3s 77ms/step - root_mean_squared_error: 0.4455 - factorized_top_k/top_1_categorical_accuracy: 2.7322e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0313 - factorized_top_k/top_10_categorical_accuracy: 0.0744 - factorized_top_k/top_50_categorical_accuracy: 0.4341 - factorized_top_k/top_100_categorical_accuracy: 0.8550 - loss: 3173.7099 - regularization_loss: 0.0000e+00 - total_loss: 3173.7099
Epoch 3/3
22/22 [==============================] - 2s 70ms/step - root_mean_squared_error: 0.444